# 1. 셀레니움 설치

In [1]:
!pip install selenium
!pip install webdriver_manager

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup
import time
import json

## 1-1. 크롬 드라이버 버전 확인

In [3]:
from selenium.webdriver.chrome.service import Service


# ChromeDriver를 사용하여 Chrome 브라우저 인스턴스 생성
driver = webdriver.Chrome()


# 버전 정보 가져오기
print("Chrome 드라이버 버전:", driver.capabilities['chrome']['chromedriverVersion'])

# 드라이버 종료
driver.quit()

Chrome 드라이버 버전: 131.0.6778.87 (ce31cae9487357cfd3bd62984ed5250121a71a23-refs/branch-heads/6778@{#2287})


# 2. url 불러오기

In [4]:
#https://linkareer.com/cover-letter/search?organizationName=삼성전자&page=1&role=DS&sort=RELEVANCE&tab=all

In [5]:
URL_TMPL = "https://linkareer.com/cover-letter/search?organizationName=삼성전자&page={}&role=DS&sort=RELEVANCE&tab=all"

cover_letter = []

driver = webdriver.Chrome()


with open("crawling_results/cover_letters2.jsonl", "w", encoding="UTF-8") as output_file:
    for page_num in range(11):
        driver.get(url=URL_TMPL.format(page_num+1))
        
        print("수집중인 페이지: [{}]".format(page_num+1))
        time.sleep(1)
        urls = driver.find_elements(By.CSS_SELECTOR, "div.MuiBox-root a") #url 저장
        for index,i in enumerate(urls):
            if index <=4: #header 태그들
                pass
            else:
                cover_letter.append({"url":i.get_attribute("href")})
                output_file.write(json.dumps({"url": i.get_attribute("href")}, ensure_ascii=False) + "\n")

driver.quit()

수집중인 페이지: [1]
수집중인 페이지: [2]
수집중인 페이지: [3]
수집중인 페이지: [4]
수집중인 페이지: [5]
수집중인 페이지: [6]
수집중인 페이지: [7]
수집중인 페이지: [8]
수집중인 페이지: [9]
수집중인 페이지: [10]
수집중인 페이지: [11]


In [6]:
print(len(cover_letter))

220


In [7]:
print(cover_letter)

[{'url': 'https://linkareer.com/cover-letter/33530?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all'}, {'url': 'https://linkareer.com/cover-letter/33943?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all'}, {'url': 'https://linkareer.com/cover-letter/33549?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all'}, {'url': 'https://linkareer.com/cover-letter/33462?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all'}, {'url': 'https://linkareer.com/cover-letter/33988?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all'}, {'url': 'https://linkareer.com/cover-letter/30466?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all'}, {'url': 'https://linkareer.com/cover-letter/33611?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEV

# 3. 본격 크롤링 진행

In [8]:
# target_divs를 제외한 텍스트를 추출
def extract_text_excluding_target(main_content):
    result_text = []
    for element in main_content.children:
        if element.name == 'div':
            continue
        if element.string:
            result_text.append(element.string.strip())
        else:
            # nested elements
            result_text.extend(extract_text_excluding_target(element))
    return result_text

### 본격적 크롤링
driver = webdriver.Chrome()

with open("crawling_results/cover_letters.jsonl", "a", encoding="UTF-8") as output_file: #기존 파일 수정
    for a,u in enumerate(cover_letter):
        driver.get(url=u["url"])
        if (a+1) %100 ==0:
            print("[{}]번째 자소서 크롤링 진행중".format(a+1))
        time.sleep(1.5)
        
        title = driver.find_element(By.CSS_SELECTOR, "h1.MuiTypography-root") #제목 저장
        spec = driver.find_element(By.CSS_SELECTOR, "h3.MuiTypography-root") #스펙 저장
        

        # 페이지 소스를 가져와서 BeautifulSoup으로 파싱
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        
        # 특정 태그를 기준으로 태그 외부의 텍스트 추출
        main_content = soup.find('article').find('main')



        # 예제에서 제외할 클래스 리스
        main_text = extract_text_excluding_target(main_content)

        cover_letter[a]['title'] = title.text
        cover_letter[a]['spec'] = spec.text
        cover_letter[a]['main_text'] = main_text
        
        
        
# 웹 드라이버 종료
driver.quit()
        


[100]번째 자소서 크롤링 진행중
[200]번째 자소서 크롤링 진행중


In [9]:
print(cover_letter)

[{'url': 'https://linkareer.com/cover-letter/33530?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all', 'title': '삼성전자 / DS 평가 및 분석 / 2024 상반기', 'spec': '서성한 / 기계공학 / 학점 4.13/4.5 / 오픽: IM2 / 사회생활 경험: 삼성전자DS 장기현장실습 / 학부연구생, 삼성전자DS 장기현장실습, 교환학생', 'main_text': ["1. 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.\n700자 이내 (영문작성 시 1400자)\n\n[대체불가능한 패키지 품질 엔지니어]\n‘당신의 위대한 상상, 우리의 가장 작은 반도체가 현실로 연결합니다.’ \n삼성전자 장기현장실습 첫 날 가장 먼저 마주하게 된, \n반도체 엔지니어의 꿈을 심어준 문구였습니다. \nTSP총괄 품질팀에서 실제 공정을 경험하고 패키지 불량 예측 및 검출 업무를 수행하며 \n반도체 사업에서 평가 및 분석 직무의 중요성을 느꼈고, \n최적의 예방품질 시스템을 구현하여 완벽한 품질의 패키지를 완성하는 \n엔지니어가 되겠다는 구체적인 목표가 생겼습니다.\n목표 달성을 위해 전문지식을 함양하고자 '나노입자공학연구실'에서 \nCMP공정에서 Silica슬러리의 SiO2막질 연마율을 높이는 연구를 진행했습니다.\n 슬러리가 타겟 막질 연마에 대한 참여율을 높일 수 있는 PH를 찾아내고, \nSi-O 결합을 더욱 쉽게 끊어내기 위해 TBAF 등의 촉매를 추가하며\n연마율을 높일 수 있는 공정 조건을 알 수 있었습니다. \n이처럼 공정 조건에 따라 크게 달라지는 막질 연마율을 보며 \n공정 최적화의 중요성을 몸소 느꼈습니다. 품질 엔지니어로서 \n반도체 전공정에 대한 이해가 필수적이라고 느껴 전기전자수업과 마이크로나노수업도 수강했습니다.\n\n삼성전자에서 세계를 주도하는 엔지니어가 되고싶

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# 크롬 드라이버 설정
driver = webdriver.Chrome()

# 페이지 열기
url = 'https://linkareer.com/cover-letter/32932?organizationName=삼성전자&page=1&role=DS&sort=RELEVANCE&tab=all'  # 여기에 실제 URL을 입력하세요
driver.get(url)

# 페이지 로딩 대기 (필요에 따라 조정)
time.sleep(5)

# 페이지 소스를 가져와서 BeautifulSoup으로 파싱
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# 특정 태그를 기준으로 태그 외부의 텍스트 추출
main_content = soup.find('article').find('main')



# 예제에서 제외할 클래스 리스
text_excluding_target = extract_text_excluding_target(main_content)

for text in text_excluding_target:
    print(text)

# 웹 드라이버 종료
driver.quit()


1. 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오. 700자 이내 (영문작성 시 1400자)

[공정과정의 마에스트로] 

저는 공정과정에서 나온 데이터를 통해 학습하고 자동화되어 스스로 판단/제어하는 스마트 팩토리를 구현해기 위해 신호 및 시스템 설계 직무에 지원했습니다. 삼성전자는 제품 시뮬레이션과 대규모 인공지능 연산을 위한 최고의 computing infra를 갖춘 곳이기에 제 꿈을 펼칠 수 있는 시작점이라고 생각하기 때문입니다. 삼성전자는 인공지능 반도체 개발 분야를 선도하고 있는 기업입니다. 이를 총괄하여 최고의 수율을 내도록 하는 단순 제조업이 아닌 지능형 공장인 스마트 팩토리는 공정과정을 전반적으로 지휘하는 마에스트로라고 느꼈습니다. 최신 기술을 항상 접해야 하고 수많은 데이터를 통해 다양한 시각을 요구하는 혁신센터는 저의 가슴을 뛰게 했습니다. 빅데이터를 이용해 시스템 요구사항을 공부하여 공정과정의 훌륭한 마에스트로가 되는 것이 최종 목표입니다.
기계공학을 전공하면서 HW의 분야에 호기심이 생겼고 컴퓨터학과를 이중전공하며 새로운 분야를 접하며 학문간의 밀접한 관계가 있다는 것을 느꼈습니다. 예를 들어 딥러닝에서 배운 CNN을 구현하여 이미지 처리, 숫자를 판별하는 프로젝트를 구현한 적이 있습니다. 이는 NPU를 설계할 때 CNN의 원리를 이용해 명령어를 설정할 때 큰 도움이 되었습니다. 이러한 경험은 신기술을 받아들일 준비와 반복적인 비효율 업무를 자동화하는 효율적인 관리 시스템을 만들 수 있어 지원하였습니다.


2. 본인의 성장과정을 간략히 기술하되 현재의 자신에게 가장 큰 영향을 끼친 사건, 인물 등을 포함하여 기술하시기 바랍니다. (※작품속 가상인물도 가능) 1500자 이내 (영문작성 시 3000자)

[Engineer, Be Curious!] 

저는 학문에 대한 끊임없는 열망과 학문 간의 융합에 대해 항상 고민하고 궁금해하는 호기심 많은 엔지니어였습니다. 이중전공을 하게 된 계기가 그렇습니다. 저는 기계공

In [11]:
def check_list_elements(lst, required_keys):
    """
    리스트 내의 각 딕셔너리를 검사하여 필요한 키가 모두 있는지, 값이 비어 있지 않은지 확인합니다.

    :param lst: 검사할 리스트
    :param required_keys: 필요한 키 리스트
    :return: 키가 누락된 요소, 값이 비어 있는 요소
    """
    missing_keys_elements = []
    empty_values_elements = []

    for idx, item in enumerate(lst):
        missing_keys = [key for key in required_keys if key not in item]
        empty_values = [key for key in required_keys if key in item and not item[key]]

        if missing_keys:
            missing_keys_elements.append((idx, missing_keys))
        if empty_values:
            empty_values_elements.append((idx, empty_values))

    return missing_keys_elements, empty_values_elements

required_keys = ['url', 'title', 'spec', 'main_text']

# 검사 실행
missing_keys_elements, empty_values_elements = check_list_elements(cover_letter, required_keys)

# 결과 출력
print("키가 누락된 요소:")
for idx, keys in missing_keys_elements:
    print(f"리스트의 {idx}번째 요소에서 누락된 키: {keys}")

print("\n값이 비어 있는 요소:")
for idx, keys in empty_values_elements:
    print(f"리스트의 {idx}번째 요소에서 비어 있는 값의 키: {keys}")


키가 누락된 요소:

값이 비어 있는 요소:
리스트의 52번째 요소에서 비어 있는 값의 키: ['spec']
리스트의 77번째 요소에서 비어 있는 값의 키: ['spec']
리스트의 89번째 요소에서 비어 있는 값의 키: ['spec']
리스트의 105번째 요소에서 비어 있는 값의 키: ['spec']
리스트의 152번째 요소에서 비어 있는 값의 키: ['spec']
리스트의 196번째 요소에서 비어 있는 값의 키: ['spec']
리스트의 199번째 요소에서 비어 있는 값의 키: ['spec']


In [12]:
print(empty_values_elements)

[(52, ['spec']), (77, ['spec']), (89, ['spec']), (105, ['spec']), (152, ['spec']), (196, ['spec']), (199, ['spec'])]


In [22]:
cover_letter[52]

{'url': 'https://linkareer.com/cover-letter/19243?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=3&role=DS&sort=RELEVANCE&tab=all',
 'title': '삼성전자 / DS / 2018 상반기',
 'spec': '',
 'main_text': ['Essay3 최근 사회이슈 중 중요하다고생각되는 한가지를 선택하고 이에 관한자신의 견해를 기술해 주시기 바랍니다.\n\n0/1000 (글자수, 공백 포함)\n\n[AI: 새로운 플랫폼의 발달]\n\n인공지능 AI 서비스는가상비서를넘어거대한플랫폼으로확대되어가고있습니다. 이세돌과의 대국으로 큰 이슈가 되었던 구글의 ‘알파고’를 시작으로아마존, KT, 카카오, 네이버등다양한분야의기업들이자신의플랫폼을기반으로한인공지능스피커를출시하며인공지능 AI의 보편화가 빠르게 이루어지고 있습니다. 이에삼성전자는갤럭시 S9 출시와 함께 ‘빅스비2.0’을선보이며빅스비를통한유비쿼터스생태계구축을예고하였습니다.\n\n이와 더불어, 인공지능 AI는 의학계에도 확대되고 있습니다. 길병원에서인공지능 AI인 ‘왓슨’을 사용하기 시작하여 치료수준이 높아지자 점차 많은 병원으로 확대되는 등 굉장히 긍정적인 반응을 얻고 있습니다. 왓슨이가진강점은과거수많은연구결과를대조분석할뿐만아니라환자의재정상황과건강상태까지고려하는등인간이고려하기힘든다양한측면에서진단을내릴수있다는것입니다. 하지만, 의학은인간의생명에직결되는요소기때문에문제상황시책임이나의학적윤리의문제가우선적으로해결되어야할것입니다.\n\n또한, 기업에서는인공지능을이용한인재채용프로그램문화가도입되고있습니다. 일본의 소프트뱅크와 NEC는작년부터 AI를 통한 서류심사를 진행하고 있고, 국내기업중마이다스아이티는인공지능인재채용프로그램인‘inAIR’를 개발하여 모든 이에게 AI면접기회를부여하고있습니다. 이러한 기업의 AI채용시스템은최근많은이슈가되었던채용비리를사전에방지하고공정성과정확성을확보할수있다는이점이있기때문에더욱

### 스펙을 아예 기재해놓지 않은 경우도 있었다

# 4. 데이터 전처리

## 4-1. 연도 전처리

In [15]:
should_fill_when = []
for a,i in enumerate(cover_letter):
    if '2015' in i['title'] and '상반기' in i['title']: ##2015~
        cover_letter[a]['when'] = '2015_상반기'
    elif '2015' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2015_하반기'
    elif '2016' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2016_상반기'
    elif '2016' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2016_하반기'
        
    elif '2017' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2017_상반기'
    elif '2017' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2017_하반기'
    elif '2018' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2018_상반기'
    elif '2018' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2018_하반기'
    elif '2019' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2019_상반기'
    elif '2019' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2019_하반기'
    elif '2020' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2020_상반기'
    elif '2020' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2020_하반기'
    elif '2021' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2021_상반기'
    elif '2021' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2021_하반기'
    elif '2022' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2022_상반기'
    elif '2022' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2022_하반기'
    elif '2023' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2023_상반기'
    elif '2023' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2023_하반기'
    elif '2024' in i['title'] and '상반기' in i['title']:
        cover_letter[a]['when'] = '2024_상반기'
    elif '2024' in i['title'] and '하반기' in i['title']:
        cover_letter[a]['when'] = '2024_하반기'
    else:
        should_fill_when.append(a)
    
    

In [16]:
print(should_fill_when) #전부 채움

[]


In [17]:
print(cover_letter[:5])

[{'url': 'https://linkareer.com/cover-letter/33530?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all', 'title': '삼성전자 / DS 평가 및 분석 / 2024 상반기', 'spec': '서성한 / 기계공학 / 학점 4.13/4.5 / 오픽: IM2 / 사회생활 경험: 삼성전자DS 장기현장실습 / 학부연구생, 삼성전자DS 장기현장실습, 교환학생', 'main_text': ["1. 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.\n700자 이내 (영문작성 시 1400자)\n\n[대체불가능한 패키지 품질 엔지니어]\n‘당신의 위대한 상상, 우리의 가장 작은 반도체가 현실로 연결합니다.’ \n삼성전자 장기현장실습 첫 날 가장 먼저 마주하게 된, \n반도체 엔지니어의 꿈을 심어준 문구였습니다. \nTSP총괄 품질팀에서 실제 공정을 경험하고 패키지 불량 예측 및 검출 업무를 수행하며 \n반도체 사업에서 평가 및 분석 직무의 중요성을 느꼈고, \n최적의 예방품질 시스템을 구현하여 완벽한 품질의 패키지를 완성하는 \n엔지니어가 되겠다는 구체적인 목표가 생겼습니다.\n목표 달성을 위해 전문지식을 함양하고자 '나노입자공학연구실'에서 \nCMP공정에서 Silica슬러리의 SiO2막질 연마율을 높이는 연구를 진행했습니다.\n 슬러리가 타겟 막질 연마에 대한 참여율을 높일 수 있는 PH를 찾아내고, \nSi-O 결합을 더욱 쉽게 끊어내기 위해 TBAF 등의 촉매를 추가하며\n연마율을 높일 수 있는 공정 조건을 알 수 있었습니다. \n이처럼 공정 조건에 따라 크게 달라지는 막질 연마율을 보며 \n공정 최적화의 중요성을 몸소 느꼈습니다. 품질 엔지니어로서 \n반도체 전공정에 대한 이해가 필수적이라고 느껴 전기전자수업과 마이크로나노수업도 수강했습니다.\n\n삼성전자에서 세계를 주도하는 엔지니어가 되고싶

In [18]:
for i in cover_letter:
    print(i['spec'])

서성한 / 기계공학 / 학점 4.13/4.5 / 오픽: IM2 / 사회생활 경험: 삼성전자DS 장기현장실습 / 학부연구생, 삼성전자DS 장기현장실습, 교환학생
연세대 / 불문, 경영 / 학점 4.1/4.3 / 오픽 AL, 토익 915, DELF B2 / 컴활 1급, GTQ 포토샵 & 일러스트 1급, 한능검 1급
홍익대학교 / 화학공학과 / 학점 4.39/4.5 / 토익: 905, 오픽: Ih / 공정실습, 데이터분석 교육
숭실대학교 / 컴퓨터학부 / 학점 3.37 / 토익: 840, 오픽: IM1 / 사회생활 경험: / 교내커뮤니티활동(학술소모임), 블로그 작성
지거국 / 전자공학부 / 학점 3.91 / IM2 / 학부연구생 6개월
숭실대 / 전기공학부 / 학점 4.15/4.5 / 토익: 900,오픽: IH / 사회생활 경험: 한창코퍼레이션 인턴 / 숭실공학상 / 한국사검정시험: 고급, 컴퓨터활용능력: , 기타: 전기기사
연세대 / 기계공학과 / 학점 3.48/4.3 / 토익: 895,오픽: AL / 사회생활 경험: 씨드로닉스(6개월), HD현대로보틱스(3개월), 넥스트랩(2개월) / 자율주행 대회 2회 / 한국사검정시험: 고급
중앙대 / 기계공학과 / 학점 4.31/4.5 / 오픽: AL / 한국사검정시험: 고급, 컴퓨터활용능력: 1급, 기타: 운전면허
홍익대학교 / 화학공학과 / 학점 4.39/4.5 / 토익: 905, 오픽: IH / 데이터분석, 공정실습
경희대 / 전자공학 / 학점 4.09/4.5 / 오픽: IH
홍익대학교 / 화학공학과 / 학점 4.39/4.5 / 토익: 945, 오픽: ih / 데이터분석, 공정실습
인서울 / 화학공학과 / 학점 3.8/4.5 / 토익: 820, 오픽: IM2
서울 하위권 4년제 / 화학공학 / 학점 4.3/4.5 / 토익스피킹: 160/AL / 반도체 장기교육 2개월
지거국 / X / 학점 3.73/4.5 / 오픽: IH / 사회생활 경험: 포스코 인턴 1개월, 한국재료연구원 현장실습 2개월 / 반도체 관련 학부연구

In [19]:
backup = cover_letter

### 4-2. spec에서 학교, 학과, 학점을 추출하여 새로운 열(column)을 구성

세세하게 나누기 어렵기 때문에 일단 spec은 전체 내용을 넣고<br>
'/'를 기준으로 나눠 첫번째를 대학교, 두번째를 학과, 세번째를 학점으로 추출했다.<br>
그러나, 만약 두번째 부분에 '.'이 들어가 있으면 학점으로 봤다.(예외처리)

In [28]:
for a, i in enumerate(cover_letter):
    parts = i['spec'].split('/')
    cover_letter[a]['univ'] = parts[0].strip()
    try:
        if '.' in parts[1]:
            cover_letter[a]['grade'] = parts[1].strip()
        else:
            cover_letter[a]['major'] = parts[1].strip()
            if '.' in parts[2]:
                cover_letter[a]['grade'] = parts[2].strip()
    except IndexError:  # parts[2]가 없을 경우를 처리
        pass

In [29]:
print(cover_letter)

[{'url': 'https://linkareer.com/cover-letter/33530?organizationName=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&page=1&role=DS&sort=RELEVANCE&tab=all', 'title': '삼성전자 / DS 평가 및 분석 / 2024 상반기', 'spec': '서성한 / 기계공학 / 학점 4.13/4.5 / 오픽: IM2 / 사회생활 경험: 삼성전자DS 장기현장실습 / 학부연구생, 삼성전자DS 장기현장실습, 교환학생', 'main_text': ["1. 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.\n700자 이내 (영문작성 시 1400자)\n\n[대체불가능한 패키지 품질 엔지니어]\n‘당신의 위대한 상상, 우리의 가장 작은 반도체가 현실로 연결합니다.’ \n삼성전자 장기현장실습 첫 날 가장 먼저 마주하게 된, \n반도체 엔지니어의 꿈을 심어준 문구였습니다. \nTSP총괄 품질팀에서 실제 공정을 경험하고 패키지 불량 예측 및 검출 업무를 수행하며 \n반도체 사업에서 평가 및 분석 직무의 중요성을 느꼈고, \n최적의 예방품질 시스템을 구현하여 완벽한 품질의 패키지를 완성하는 \n엔지니어가 되겠다는 구체적인 목표가 생겼습니다.\n목표 달성을 위해 전문지식을 함양하고자 '나노입자공학연구실'에서 \nCMP공정에서 Silica슬러리의 SiO2막질 연마율을 높이는 연구를 진행했습니다.\n 슬러리가 타겟 막질 연마에 대한 참여율을 높일 수 있는 PH를 찾아내고, \nSi-O 결합을 더욱 쉽게 끊어내기 위해 TBAF 등의 촉매를 추가하며\n연마율을 높일 수 있는 공정 조건을 알 수 있었습니다. \n이처럼 공정 조건에 따라 크게 달라지는 막질 연마율을 보며 \n공정 최적화의 중요성을 몸소 느꼈습니다. 품질 엔지니어로서 \n반도체 전공정에 대한 이해가 필수적이라고 느껴 전기전자수업과 마이크로나노수업도 수강했습니다.\n\n삼성전자에서 세계를 주도하는 엔지니어가 되고싶

In [40]:
for i in range(5):
    print(cover_letter[i]['univ']+";"+cover_letter[i]['major']+";"+cover_letter[i]['grade'])

서성한;기계공학;학점 4.13
연세대;불문, 경영;학점 4.1
홍익대학교;화학공학과;학점 4.39
숭실대학교;컴퓨터학부;학점 3.37
지거국;전자공학부;학점 3.91


In [41]:
# jsonl 파일에 작성
with open('crawling_results/cover_letters.jsonl', 'w', encoding='utf-8') as file:
    for item in cover_letter:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

In [42]:
import pandas as pd

# JSONL 파일 경로
file_path = 'crawling_results/cover_letters.jsonl'

# JSONL 파일을 데이터프레임으로 읽어오기
try:
    df = pd.read_json(file_path, lines=True, encoding='utf-8')
    print("JSONL 파일이 성공적으로 데이터프레임으로 로드되었습니다.")
except ValueError as e:
    print(f"JSONL 파일을 읽는 중 오류가 발생했습니다: {e}")
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {file_path}")

# 데이터프레임 확인 (선택 사항)
print(df.head())

# 필요에 따라 데이터프레임을 CSV 파일로 저장할 수도 있습니다.
# df.to_csv('crawling_results/cover_letters.csv', index=False, encoding='utf-8')

JSONL 파일이 성공적으로 데이터프레임으로 로드되었습니다.
                                                 url  \
0  https://linkareer.com/cover-letter/33530?organ...   
1  https://linkareer.com/cover-letter/33943?organ...   
2  https://linkareer.com/cover-letter/33549?organ...   
3  https://linkareer.com/cover-letter/33462?organ...   
4  https://linkareer.com/cover-letter/33988?organ...   

                                        title  \
0                삼성전자 / DS 평가 및 분석 / 2024 상반기   
1        삼성전자 / DS 부문 메모리사업부 영업마케팅 / 2024 하반기   
2        삼성전자 / DS 제조&기술담당_반도체공정기술 / 2024 상반기   
3                 삼성전자 / DS부문_SW개발 / 2023 하반기   
4  삼성전자 / DS 부문 System LSI사업부_SW개발 / 2024 하반기   

                                                spec  \
0  서성한 / 기계공학 / 학점 4.13/4.5 / 오픽: IM2 / 사회생활 경험: ...   
1  연세대 / 불문, 경영 / 학점 4.1/4.3 / 오픽 AL, 토익 915, DEL...   
2  홍익대학교 / 화학공학과 / 학점 4.39/4.5 / 토익: 905, 오픽: Ih ...   
3  숭실대학교 / 컴퓨터학부 / 학점 3.37 / 토익: 840, 오픽: IM1 / 사...   
4            지거국 / 전자공학부 / 학점 3.91 / IM2 / 학부연구생 6개월  

In [43]:
print(len(df))

220
